# Set Your Data Free with Kafka - SQLCLR

In [ ]:
USE GamePlayDB;

In [ ]:
SELECT DB_NAME();

## Clean Up

Let's clean up and drop procedures and assemblies.

In [ ]:
IF EXISTS(SELECT 1 FROM sys.procedures WHERE name LIKE('%pr_PublishToKafkaCLR%'))
BEGIN
  DROP PROCEDURE dbo.pr_PublishToKafkaCLR;
END

In [ ]:
IF EXISTS(SELECT 1 FROM sys.assemblies WHERE name LIKE('%KafkaProducer%'))
BEGIN
  DROP ASSEMBLY KafkaProducer;
END

## Create Assembly and Procs

### Assembly

Replace the `<full-path-to>` below with the path to the Kafka producer dll. You can find it in the download under `..\code\bin\sqlclr\`. The customized Confluent Kafka client dll is also in the same location and will automatically be deployed when you call `CREATE ASSEMBLY ...`.

In [ ]:
CREATE ASSEMBLY KafkaProducer
FROM '<full-path-to>\KafkaSqlClrProducer.dll'
WITH PERMISSION_SET = UNSAFE;

In [ ]:
SELECT * FROM sys.assemblies;

### Procedure

Create a proc against the `PostMessage` method in the `KafkaProducer` class. The signature looks like so:
`PostMessage(string bootStrapServers, string topic, string libRdPath, string partitionKey, string value)`.

In [ ]:
CREATE OR ALTER PROCEDURE dbo.pr_PublishToKafkaCLR  @BootStrapServers nvarchar(256),
                                          @Topic nvarchar(50),
                                          @LibRdPath nvarchar(1024),
                                          @PartitionKey nvarchar(126),
                                          @Message nvarchar(max)
AS EXTERNAL NAME [KafkaProducer].[KafkaSqlClrProducer.KafkaProducer].[PostMessage];


Clean out all messages in the `testTopic` and test the proc. Replace the `<full-path-to>` in the `@LibRdPath` parameter with the path to where `librdkafka.dll` is. You find it in the download under `..\code\bin\sqlclr\librdkafka`. 

Before you go on executing the code below, make sure you have the two topics; `testTopic`, and `wagers` in Kafka.

When you have executed the code below, check in **Confluent Control Center** that you have an event in the `testTopic` topic.

In [ ]:
DECLARE @offset int;
EXEC @offset = dbo.pr_PublishToKafkaCLR  @BootStrapServers = N'localhost:9092'
                         , @Topic = N'testTopic'
                         , @LibRdPath = N'<full-path-to>\librdkafka.dll'
                         , @PartitionKey = N'1'
                         , @Message = N'Hello World'
SELECT @offset Offset;
 

## Hook Up Wagering

### Hook-point Procedure

This procedure generates the event from the `dbo.pr_LogWager` proc. Change the `@LibRdPath` parameter below to point to where the `librdkafka.dll` file is. You find it in the download under `..\code\bin\sqlclr\librdkafka`. Oh, the`librdkafka.dll` requires the other files in the `librdkafka` directory, so do not delete those.

In [ ]:
CREATE OR ALTER PROCEDURE dbo.pr_GenerateAndPublishWagerEvent @UserID INT,
                                                              @GameID INT,
                                                              @WagerAmount bigint,
                                                              @PayoutAmount bigint,
                                                              @EventTime datetime2,
                                                              @eventID int
AS
BEGIN
  DECLARE @msg nvarchar(max),
          @servers nvarchar(256),
          @topic NVARCHAR(50);

  --get settings
  SELECT @servers = kc.BootstrapServers, @topic = kes.Topic
  FROM dbo.tb_KafkaCluster kc
  JOIN dbo.tb_KafkaEventSubscriber kes
    ON kc.ClusterID = kes.ClusterID
  WHERE kes.EventID =  @eventID

  --generate the event
  SET @msg =  (SELECT 1500 AS eventTypeId,
                 @UserId AS userId,
                 @GameId AS gameId,
                 @WagerAmount AS wagerAmount,
                 @PayoutAmount AS payoutAmount,
                 @EventTime AS eventTime
     FOR JSON PATH, WITHOUT_ARRAY_WRAPPER);

  EXEC dbo.pr_PublishToKafkaCLR  @BootStrapServers = @servers
                                , @Topic = @topic
                                , @LibRdPath = N'<full-path-to>\librdkafka.dll'
                                , @PartitionKey = @UserID
                                , @Message = @msg
END

Hook in the proc into the `dbo.pr_LogWager` proc

In [ ]:
CREATE OR ALTER PROCEDURE dbo.pr_LogWager @UserID INT,
                                          @GameID INT,
                                          @WagerAmount bigint,
                                          @PayoutAmount bigint,
                                          @EventTime datetime2
AS
BEGIN
  IF(@EventTime IS NULL)
  BEGIN
    SET @EventTime = SYSUTCDATETIME();
  END 

  BEGIN TRY
    BEGIN TRAN
      INSERT INTO dbo.tb_GamePlay(UserID, GameID, WagerAmount, PayoutAmount, EventTime)
      VALUES(@UserID, @GameID, @WagerAmount, @PayoutAmount, @EventTime);
      --do some more tx "stuff" here
    COMMIT TRAN

    EXEC dbo.pr_GenerateAndPublishWagerEvent @UserID = @UserID,
                                              @GameID = @GameID,
                                              @WagerAmount = @WagerAmount,
                                              @PayoutAmount = @PayoutAmount,
                                              @EventTime = @EventTime,
                                              @eventID = 1500;
                              
  END TRY
  BEGIN CATCH
    ROLLBACK TRAN;
  END CATCH

END  
GO

### Test
Don't forget to create the `wagers` topic.

In [ ]:
SET NOCOUNT ON;
EXECUTE dbo.pr_EmulateGamePlay @Loops = 3;

In [ ]:
SELECT * FROM dbo.tb_GamePlay

When you have executed the code go to **Confluent Control Center** and ensure you have data in the `wagers` topic.

Finally, just clean out the `dbo.tb_GamePlay` table.

In [ ]:
TRUNCATE TABLE dbo.tb_GamePlay